In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R

scratch_path <- Sys.getenv("SCRATCH")
.libPaths(file.path(scratch_path, "Rlocal4.3.2"))

In [ ]:
%%R

library(DESeq2)
library(magrittr)
library(SummarizedExperiment)

IN_DIR <- "../../inputs/"
OUT_DIR <- "../../outputs/"

In [ ]:
%%R

dds <- readRDS(paste(OUT_DIR,"dds.Rds",sep=""))
print(dds)

In [ ]:
%%R

vst.counts <- DESeq2::vst(dds,
                          blind = FALSE,
                          fitType = "local")
print(vst.counts)

In [ ]:
%%R

print(colData(vst.counts))

In [ ]:
%%R

saveRDS(vst.counts,
        paste(OUT_DIR,"vst_counts.Rds",sep=""))

In [ ]:
%%R

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")
head(ensembl2rxns.df, n=5)

In [ ]:
%%R

reactome_ensembl_ids <- readRDS(paste(OUT_DIR,"reactome_ensembl_ids.Rds",sep=""))
print(reactome_ensembl_ids)

In [ ]:
%%R

rxn2ensembls.nls <- list()
rxns_w_GEO_model_training_ensembls.df <- ensembl2rxns.df %>% dplyr::filter(V1 %in% reactome_ensembl_ids)
print(rxns_w_GEO_model_training_ensembls.df)

In [ ]:
%%R

rxns_w_GEO_model_training_ensembls.df$V1 <- as.character(rxns_w_GEO_model_training_ensembls.df$V1)
rxns_w_GEO_model_training_ensembls.df$V2 <- as.character(rxns_w_GEO_model_training_ensembls.df$V2)
for(i in 1:nrow(rxns_w_GEO_model_training_ensembls.df)){
  ens_id <- rxns_w_GEO_model_training_ensembls.df$V1[i]
  rxn_id <- rxns_w_GEO_model_training_ensembls.df$V2[i]
  ensembl_list_for_rxn_id <- rxn2ensembls.nls[[rxn_id]]
  if(is.null(ensembl_list_for_rxn_id)){
    ensembl_list_for_rxn_id <- c(ens_id)
  }
  rxn2ensembls.nls[[rxn_id]] <- c(ensembl_list_for_rxn_id,ens_id) %>% unique()
}
saveRDS(rxn2ensembls.nls,
        paste(OUT_DIR,"rxn2ensembls_nls.Rds",sep=""))

In [ ]:
%%R
length(rxn2ensembls.nls)